In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=UserWarning)

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/JackyP/testing/master/datasets/nycflights.csv", index_col=0)
df["data"] = pd.to_datetime(df[['year', 'month', 'day']]) 
df.head()

,year,month,day,dep_time,dep_delay,arr_time,arr_delay,carrier,tailnum,flight,origin,dest,air_time,distance,hour,minute,data
1,2013,1,1,517.0,2.0,830.0,11.0,UA,N14228,1545,EWR,IAH,227.0,1400,5.0,17.0,2013-01-01
2,2013,1,1,533.0,4.0,850.0,20.0,UA,N24211,1714,LGA,IAH,227.0,1416,5.0,33.0,2013-01-01
3,2013,1,1,542.0,2.0,923.0,33.0,AA,N619AA,1141,JFK,MIA,160.0,1089,5.0,42.0,2013-01-01
4,2013,1,1,544.0,-1.0,1004.0,-18.0,B6,N804JB,725,JFK,BQN,183.0,1576,5.0,44.0,2013-01-01
5,2013,1,1,554.0,-6.0,812.0,-25.0,DL,N668DN,461,LGA,ATL,116.0,762,5.0,54.0,2013-01-01


### 1. Será que tem uma companhia que atrasa mais ou menos? Será que tem relação com mês?

*MAPA DE CALOR: Compara valores de um indicador numérico entre grupos baseados em outros 2 indicadores, por meio de uma matriz de cores.*

In [ ]:
tmp = df.pivot_table(index='month', columns='carrier', values="arr_delay", aggfunc="mean")
tmp.fillna(0, inplace=True) 
plt.figure(figsize=(12, 6))
sns.heatmap(tmp, linecolor='white', linewidth=1, cmap="Pastel1")
plt.title('Quantidade médio de atraso de chegada por companhia e mês');

### 2. Quais são as origens mais frequentes?  E os destinos?

Gráfico de setores : Representa frequências por grupo, tal como o gráfico de barras; mas a comparação visual dos grupos é prejudicada por envolver os aspectos de área e ângulo, especialmente se as distribuições forem semelhantes.

In [ ]:
plt.pie(
    dict(df['origin'].value_counts()).values(),
    labels=dict(df['origin'].value_counts()).keys()
    );

In [ ]:
df.groupby("origin").count()

In [ ]:
# A partir de um dataframe
tmp = pd.DataFrame(df.groupby("origin").count())
tmp.plot.pie(y='day',
             autopct='%1.2f%%',
             #explode = (0.1, 0, 0),
             ylabel='',
             fontsize=10,
             figsize=(8,6)
             );

Gráfico de roscas : Versão alternativa do gráfico de setores, um pouco mais pautada na noção de comprimento (de arcos)


In [ ]:
tmp.plot.pie(y='day',
             autopct='%1.2f%%',
             ylabel='',
             explode = (0.1, 0, 0),
             pctdistance=0.8, # controla a posição do texto
             fontsize=10,
             figsize=(8,6),
             title='Distribuição da origem dos vôos');
circle = plt.Circle( (0,0), 0.5, color='white')
p=plt.gcf()
p.gca().add_artist(circle);

In [ ]:
#plt.figure(figsize=(20, 12))

plt.pie(dict(df['dest'].value_counts()).values(),
         labels = dict(df['dest'].value_counts()).keys());

In [ ]:
# Melhorando as legendas
plt.pie(dict(df['dest'].value_counts()).values())
plt.legend(labels = dict(df['dest'].value_counts()).keys(),
           loc='upper right', ncol = 5, bbox_to_anchor=[2.5, 1], fontsize=9);

### 3. Será que tem uma companhia que atrasa (chegada e partida) mais ou menos? 

Gráfico de bolhas :  Semelhante a um gráfico de dispersão, mas incorpora um terceiro indicador numérico, representado pelas áreas das bolhas

In [ ]:
tmp = df.groupby(["carrier"])[["dep_delay", "arr_delay"]].mean().reset_index()
sns.set_theme(style="darkgrid")

sns.scatterplot(
    data=tmp,
    x="carrier",
    y="arr_delay",
    size="dep_delay",
    legend=False,
    sizes=(20, 2000)
);

### 4. Representatividade das companhias

Barra simples, utilizado para :
- Representar frequências absolutas ou relativas (%) associadas às categorias de um indicador.
- Comparar valores de um indicador numérico com quebra de grupos.
- Evolução de um indicador numérico ao longo do tempo.

In [ ]:
plt.bar(
    list(dict(df['carrier'].value_counts()).keys()),
    list(dict(df['carrier'].value_counts()).values())
    );

In [ ]:
fig, ax = plt.subplots(figsize = (12,4))
ax.bar(
    list(dict(df['carrier'].value_counts()).keys()),
    list(dict(df['carrier'].value_counts()).values()),
    color = "#4CAF50",
    width=.5,
    )
ax.bar_label(ax.containers[0], label_type='edge', color = "blue", rotation = 90)
ax.set_ylabel('Quantidade de vôos')
ax.set_title('Quantidade de vôos por companhia');

### 5. Será que as companhias que voam mais, voam por mais tempo? Será que tem companhia que só voa em uma origem?

Barras empilhadas:
-  Representação de frequências relacionadas a 2+ grupos, ao longo do tempo ou não.
-  Pode-se representar frequências relativas (%), de forma que todas as barras somem 100%


In [ ]:
tmp = df.pivot_table(index='carrier', columns='origin', values='air_time', aggfunc="sum").reset_index()
tmp.plot(x='carrier', kind='barh', stacked=True, 
         title='Tempo de vôo por companhia', 
         width=.9,
         );

Barra simples agrupado: podem ser úteis quando há 2 ou mais públicos ou indicadores para comparar

In [ ]:
tmp.plot(x='carrier', kind='bar', stacked=False, 
         title='Tempo de vôo por companhia', 
         figsize=(15,4),
         width=.7,
         ylim=(0, 10000000) # ajuste para ficar com o mesmo limite do de cima
         );

### 6. Será que a origem interfere nos atrasos?
Gráfico de linhas
-  Ideal para representar evolução de um indicador numérico ao longo do tempo, devido à noção de direção: aumento/redução.
- Pode ser utilizado para comparar a evolução por grupo, desde que não haja muitas intersecções entre as linhas.
-  Funciona melhor que o gráfico de barras para evoluções, especialmente quando há alta quantidade de pontos.

In [ ]:
tmp = df.groupby(["data","origin"])["dep_delay"].mean().reset_index()
plt.figure(figsize=(15,4))
plt.title("Atraso médio por origem")
sns.lineplot(data=tmp, x="data",y='dep_delay', hue='origin');

### 7. Quem atrasa para partir atrasa para chegar?

Gráficos de dispersão: Utilizado para avaliar relações entre dois indicadores numéricos

In [ ]:
sns.set_style("whitegrid")
ax = sns.scatterplot(data=df, x="arr_delay", y="dep_delay")
ax.set_title("Relação entre atraso de partida e chegada");